In [1]:
subj_id = 1


In [2]:
import pandas as pd
import numpy as np
# 读取原始的 TSV 文件
df = pd.read_csv('/root/autodl-tmp/NSD/nsddata_betas/nsd_stim_info_merged.csv', delimiter=',')

# data = df[df[f'subject{subj_id}']==1]
train_data = df[(df['shared1000']==False) & (df[f'subject{subj_id}']==1)]
val_data = df[(df['shared1000']==True) & (df[f'subject{subj_id}']==1)]



train_idx = train_data[[f'subject{subj_id}_rep{i}' for i in range(3)]].values
val_idx = val_data[[f'subject{subj_id}_rep{i}' for i in range(3)]].values
train_image_idx = train_data['nsdId'].values
val_image_idx = val_data['nsdId'].values

np.save(f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/subj{subj_id:02}_train_index.npy',train_idx)
np.save(f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/subj{subj_id:02}_train_image_index.npy',train_image_idx)
np.save(f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/subj{subj_id:02}_val_index.npy',val_idx)
np.save(f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/subj{subj_id:02}_val_image_index.npy',val_image_idx)


In [3]:
print(train_idx.shape)
print(val_idx.shape)

(9000, 3)
(1000, 3)


In [4]:
len(df)

73000

In [9]:
import os
from tqdm import tqdm
import nibabel as nb
import numpy as np

def read_nii_files_in_directory(directory):
    for filename in tqdm(os.listdir(directory)):
        if filename.endswith('.nii.gz'):
            file_path = os.path.join(directory, filename)
            nii = nb.load(file_path)
            img = np.asarray(nii.dataobj)
            # 保存数据到.npy文件
            output_file = f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/numpy/{filename.split(".")[0]}.npy'
            np.save(output_file, img)

            

# 指定文件夹路径
directory_path = f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/func1pt8mm/betas_fithrf_GLMdenoise_RR'
read_nii_files_in_directory(directory_path)

100%|██████████| 42/42 [03:32<00:00,  5.06s/it]


In [6]:
import gc
from tqdm import tqdm
import numpy as np

data = np.ndarray(shape = (30000,81,104,83),dtype='float16')
start=0
for i in tqdm(range(1,41)):
    data0 = np.load(f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/numpy/betas_session{i:02}.npy')
    # data0 [x,y,z,b]
    mean = data0.mean(3,keepdims = True)
    std = data0.std(3,keepdims = True)
    data0 = (data0-mean)/std
    data0[np.isnan(data0)]=0
    # 不知道为什么要每个Session做标准化，但是这里的问题是没有排除test数据集
    l=data0.shape[3] # 750
    end = start+l
    data[start:end,:,:,:] = data0.transpose((3,0,1,2))
    start=end
    del data0
    gc.collect()

np.save(f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/subj{subj_id:02}_fmri_ori.npy',data)

  0%|          | 0/40 [00:00<?, ?it/s]/tmp/ipykernel_1065/3916499870.py:12: RuntimeWarning: invalid value encountered in divide
  data0 = (data0-mean)/std
100%|██████████| 40/40 [16:19<00:00, 24.48s/it]


In [4]:
# 新版本不用跑下面两个分割

In [3]:
import numpy as np
import gc
# index = np.arange(10000)
# np.random.shuffle(index)
# train_idx = index[:9000]
# val_idx = index[9000:10000]
# fmri_path = f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/subj{subj_id:02}_fmri.npy'
# fmri = np.load(fmri_path, mmap_mode='r')
ori_fmri = np.load(f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/subj{subj_id:02}_fmri_ori.npy',mmap_mode='r')

train_idx = np.load(f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/subj{subj_id:02}_train_index.npy')
val_idx = np.load(f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/subj{subj_id:02}_val_index.npy')

train_fmri = ori_fmri[train_idx-1]
np.save(f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/new_split/train_fmri.npy',train_fmri)
del train_fmri
gc.collect()

val_fmri = ori_fmri[val_idx-1]
np.save(f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/new_split/val_fmri.npy',val_fmri)


del val_fmri
del ori_fmri

gc.collect()



0

In [ ]:
# image_path = f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/subj{subj_id:02}_images.npy'
# images = np.load(image_path, mmap_mode='r')
# the image file
import h5py
import numpy as np
import gc
with h5py.File('/root/autodl-tmp/NSD/nsd/nsd_stimuli.hdf5') as image_file:
    # print("Datasets in HDF5 file:", list(hdf_file.keys())) # ['imgBrick']
    images = image_file['imgBrick']

    train_image_idx = np.load(f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/subj{subj_id:02}_train_image_index.npy')
    train_images = images[train_image_idx]
    val_image_idx = np.load(f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/subj{subj_id:02}_val_image_index.npy')

    np.save(f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/new_split/train_images.npy',train_images)
    del train_images
    gc.collect()

    val_images = images[val_image_idx]
    np.save(f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/new_split/val_images.npy',val_images)


In [4]:
train_image_idx

array([   13,    27,    71, ..., 72985, 72992, 72999])

In [5]:
import numpy as np
data = np.load('/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj01/new_split/val_fmri.npy', mmap_mode='r')

In [6]:
print(data.shape)

(1000, 3, 81, 104, 83)


# 将ori文件分割后重新保存

In [1]:
import numpy as np
oridata = np.load(f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj:02}/subj{subj:02}_fmri_ori.npy',mmap_mode='r')

# oridata.shape
data = np.zeros((30000,60,40,40)).astype('float16')

chunk_size = 3000
total = 30000
for start_i in range(0,total,chunk_size):
    end_i = start_i+chunk_size
    chunk =  oridata[start_i:end_i,12:72,:40,20:60]
    data[start_i:end_i] = chunk
    del chunk
np.save(f'/root/autodl-tmp/NSD/nsddata_betas/ppdata/subj{subj_id:02}/subj{subj_id:02}_fmri_resize.npy',data.astype('float16'))

NameError: name 'subj' is not defined